In [ ]:
"""
Cryptarithm Solver using Z3 SMT Solver
Solves puzzles like SEND + MORE = MONEY
"""

from z3 import Solver, Int, Distinct, sat
import random


def generate_cryptarithm():
    """
    Generate a valid cryptarithm puzzle.
    Returns: (operand1, operand2, result) as strings

    This function creates a random addition puzzle where:
    - Each letter maps to a unique digit (0-9)
    - Leading letters cannot be 0
    - The arithmetic is valid
    """
    # Word lists for generating puzzles
    words_3 = ["CAT", "DOG", "BAT", "RAT", "HAT", "SUN", "RUN", "FUN"]
    words_4 = ["SEND", "MAKE", "TAKE", "GIVE", "LOVE", "HOPE"]
    words_5 = ["MONEY", "WORLD", "EARTH", "MAGIC"]

    # Try to generate a valid puzzle
    for _ in range(100):
        # Randomly select word lengths
        if random.choice([True, False]):
            op1 = random.choice(words_4)
            op2 = random.choice(words_4)
        else:
            op1 = random.choice(words_3)
            op2 = random.choice(words_3)

        # Get all unique letters
        letters = list(set(op1 + op2))
        if len(letters) > 10:
            continue

        # Create result word (usually one letter longer)
        result_len = max(len(op1), len(op2)) + 1

        # Try to form a result word
        available_letters = [c for c in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" if c not in letters]
        if len(available_letters) < result_len - len(letters):
            continue

        # Build result from existing and new letters
        result = letters[0]  # Start with a letter from operands
        needed = result_len - 1
        result += ''.join(random.sample(available_letters, min(needed, len(available_letters))))

        # Pad if needed
        while len(result) < result_len:
            result += random.choice(letters)

        # Check if this puzzle is solvable
        solution = solve_cryptarithm(op1, op2, result)
        if solution:
            return op1, op2, result

    # Fallback to known valid puzzle
    return "SEND", "MORE", "MONEY"


def solve_cryptarithm(operand1, operand2, result):
    """
    Solve a cryptarithm puzzle using Z3.

    Args:
        operand1: First operand as string (e.g., "SEND")
        operand2: Second operand as string (e.g., "MORE")
        result: Result as string (e.g., "MONEY")

    Returns:
        Dictionary mapping letters to digits, or None if no solution
    """
    # Get all unique letters
    letters = list(set(operand1 + operand2 + result))

    # Create Z3 integer variables for each letter
    letter_vars = {letter: Int(letter) for letter in letters}

    # Create Z3 solver
    solver = Solver()

    # Constraint 1: Each letter is a digit (0-9)
    for letter in letters:
        solver.add(letter_vars[letter] >= 0, letter_vars[letter] <= 9)

    # Constraint 2: All letters map to different digits
    solver.add(Distinct([letter_vars[letter] for letter in letters]))

    # Constraint 3: Leading letters cannot be 0
    leading_letters = {operand1[0], operand2[0], result[0]}
    for letter in leading_letters:
        solver.add(letter_vars[letter] != 0)

    # Helper: Convert word to numeric Z3 expression
    def word_to_number(word):
        return sum(letter_vars[ch] * (10 ** (len(word) - i - 1)) for i, ch in enumerate(word))

    # Arithmetic constraint
    solver.add(word_to_number(operand1) + word_to_number(operand2) == word_to_number(result))

    # Check if solution exists
    if solver.check() == sat:
        model = solver.model()
        return {letter: model[letter_vars[letter]].as_long() for letter in letters}
    else:
        return None


def display_cryptarithm(operand1, operand2, result, solution=None):
    """
    Display the cryptarithm puzzle and optionally its solution.
    """
    max_len = max(len(operand1), len(operand2), len(result))

    print("\n" + "=" * 50)
    print("CRYPTARITHM PUZZLE")
    print("=" * 50)

    # Display puzzle
    print(f"\n{operand1:>{max_len}}")
    print(f"+ {operand2:>{max_len - 2}}")
    print("-" * (max_len + 2))
    print(f"= {result:>{max_len}}")

    if solution:
        print("\n" + "=" * 50)
        print("SOLUTION")
        print("=" * 50)

        # Display letter-to-digit mapping
        print("\nLetter Mappings:")
        for letter in sorted(solution.keys()):
            print(f"  {letter} = {solution[letter]}")

        # Convert words to numbers
        def word_to_num(word):
            return int(''.join(str(solution[ch]) for ch in word))

        num1 = word_to_num(operand1)
        num2 = word_to_num(operand2)
        num_result = word_to_num(result)

        # Display numeric solution
        print(f"\nNumeric Solution:")
        print(f"{num1:>{max_len}}")
        print(f"+ {num2:>{max_len - 2}}")
        print("-" * (max_len + 2))
        print(f"= {num_result:>{max_len}}")

        # Verify
        if num1 + num2 == num_result:
            print("\n✓ Solution verified correct!")
        else:
            print("\n✗ Solution verification failed!")

    print("=" * 50 + "\n")


def test_cryptarithm_solver():
    """
    Test the cryptarithm solver with various test cases.
    """
    print("TESTING CRYPTARITHM SOLVER")
    print("=" * 50)

    # Test Case 1: Classic SEND + MORE = MONEY
    print("\nTest Case 1: SEND + MORE = MONEY")
    solution1 = solve_cryptarithm("SEND", "MORE", "MONEY")
    display_cryptarithm("SEND", "MORE", "MONEY", solution1)

    # Test Case 2: Simple puzzle
    print("\nTest Case 2: AB + AB = CBC")
    solution2 = solve_cryptarithm("AB", "AB", "CBC")
    display_cryptarithm("AB", "AB", "CBC", solution2)

    # Test Case 3: Unsolvable puzzle
    print("\nTest Case 3: ABC + DEF = GH (Unsolvable)")
    solution3 = solve_cryptarithm("ABC", "DEF", "GH")
    if solution3:
        display_cryptarithm("ABC", "DEF", "GH", solution3)
    else:
        print("No solution exists for this puzzle.")

    # Test Case 4: Edge case - carry propagation
    print("\nTest Case 4: AAA + BBB = CCCC")
    solution4 = solve_cryptarithm("AAA", "BBB", "CCCC")
    display_cryptarithm("AAA", "BBB", "CCCC", solution4)


def main():
    """
    Main function to demonstrate cryptarithm solver.
    """
    print("\n" + "=" * 50)
    print("CRYPTARITHM SOLVER USING Z3")
    print("=" * 50)

    # Run predefined tests
    test_cryptarithm_solver()

    # Generate and solve a random puzzle
    print("\n" + "=" * 50)
    print("GENERATED RANDOM PUZZLE")
    print("=" * 50)
    op1, op2, res = generate_cryptarithm()
    solution = solve_cryptarithm(op1, op2, res)
    display_cryptarithm(op1, op2, res, solution)


if __name__ == "__main__":
    main()



CRYPTARITHM SOLVER USING Z3
TESTING CRYPTARITHM SOLVER

Test Case 1: SEND + MORE = MONEY

CRYPTARITHM PUZZLE

 SEND
+ MORE
-------
= MONEY

SOLUTION

Letter Mappings:
  D = 7
  E = 5
  M = 1
  N = 6
  O = 0
  R = 8
  S = 9
  Y = 2

Numeric Solution:
 9567
+ 1085
-------
= 10652

✓ Solution verified correct!


Test Case 2: AB + AB = CBC

CRYPTARITHM PUZZLE

 AB
+ AB
-----
= CBC


Test Case 3: ABC + DEF = GH (Unsolvable)
No solution exists for this puzzle.

Test Case 4: AAA + BBB = CCCC

CRYPTARITHM PUZZLE

 AAA
+ BBB
------
= CCCC


GENERATED RANDOM PUZZLE

CRYPTARITHM PUZZLE

 RUN
+ FUN
------
= NXOG

SOLUTION

Letter Mappings:
  F = 7
  G = 2
  N = 1
  O = 8
  R = 3
  U = 4
  X = 0

Numeric Solution:
 341
+ 741
------
= 1082

✓ Solution verified correct!



In [ ]:
!pip install z3

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.0 MB/s eta 0:00:00


In [ ]:
!pip install z3-solver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 69.3 MB/s eta 0:00:00
